In [87]:
from requests import post, get
import base64

In [38]:
client_id = "your-client-id"

client_secret = "your-client-secret"

url = "https://accounts.spotify.com/api/token"


def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
    }

    data = {
            "grant_type": "client_credentials"
            }

    result = post(url, headers=headers, data=data)

    return result.json()["access_token"]



In [39]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [96]:
def search_for_anything(token, artist_name, type):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"q={artist_name}&type={type}&limit=1"

    query_url = url + "?" + query 
    
    result = get(query_url, headers=headers)
    
    return result.json()


In [77]:
def list_playlist(token, playlist_id):
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields=items%28added_at%2Ctrack%28name%2Crelease_date%2C+artists%28name%29%2C+duration_ms%29%29'
    headers = get_auth_header(token)
    
    result = get(url, headers=headers)

    return result.json()

In [71]:
token = get_token()

In [79]:


#print(search_for_anything(token, "Gusttavo Lima", type="artist"))
#
print(search_for_anything(token, "Radio Birita", type="playlist"))
#
#print(search_for_anything(token, "Joen Rogan", type="artist"))
#
#print(search_for_anything(token, "David Goggings", type="episode"))

#print(search_for_anything(token, "The Joe Rogan Experience", type="playlist"))

print(list_playlist(token, "16FFBX4MVut2qVV4SnTpZr"))


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=Radio+Birita&type=playlist&offset=0&limit=1', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/16FFBX4MVut2qVV4SnTpZr'}, 'href': 'https://api.spotify.com/v1/playlists/16FFBX4MVut2qVV4SnTpZr', 'id': '16FFBX4MVut2qVV4SnTpZr', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000bebbe091b2369067df11f06d888b', 'width': None}], 'name': 'RÁDIO BIRITA', 'owner': {'display_name': 'Bárbara Lemos', 'external_urls': {'spotify': 'https://open.spotify.com/user/12160624005'}, 'href': 'https://api.spotify.com/v1/users/12160624005', 'id': '12160624005', 'type': 'user', 'uri': 'spotify:user:12160624005'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTc4MSw0Y2FkOTY5OTA4YTNlMjFmYzY0ZTVmMjIyNmY1NGJhM2RmODM5MGI3', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/16FFBX4MVut2qVV4SnTpZr/tracks', 'total': 165}, 'type': 'pla

In [114]:
playlist = search_for_anything(token, "Radio Birita", type="playlist")
id_playlist = playlist['playlists']['items'][0]['id']

songs = list_playlist(token, id_playlist)
type(songs)


dict

,added_at,track
0,2023-08-18T17:35:59Z,"{'artists': [{'name': 'Ícaro e Gilmar'}, {'nam..."
1,2023-08-18T17:21:57Z,"{'artists': [{'name': 'Bruno & Rafa'}, {'name'..."
2,2023-09-11T04:32:40Z,"{'artists': [{'name': 'Gusttavo Lima'}, {'name..."
3,2023-06-24T16:47:45Z,"{'artists': [{'name': 'Humberto & Ronaldo'}], ..."
4,2023-09-11T04:36:34Z,"{'artists': [{'name': 'Fernando & Sorocaba'}, ..."
